In [1]:
import pandas as pd
import re
import os

# Extracting Information

In [2]:
# read skillsfuture.txt
with open(r"../skillsfuture.txt", 'r', encoding="utf-8") as file:
    # Read the entire content of the file as a single string
    target_text = file.read()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x85 in position 578696: invalid start byte

In [17]:
def extract_body(input_text):
    try:
        start_index = re.search(r"""<div class="card-body">""", input_text).start()
        end_index = re.search(r"""<div class="card-footer">""", input_text).start()
        return input_text[start_index: end_index]
    except:
        return None

def extract_footer(input_text):
    try:
        start_index = re.search(r"""<div class="card-footer">""", input_text).start()
        end_index = re.search(r"""netspin""", input_text).start()
        return input_text[start_index: end_index]
    except:
        return None

def extract_course_provider(input_text, working=True):
    """
    extract the following:
    - course provider
    """
    try:
        start_index =  re.search("""<div class="course-provider">""", input_text).start()
        end_index = re.search("""<!-- Course Title -->""", input_text).start()
    except:
        return None
    temp_text = input_text[start_index:end_index]
    if working:
        return temp_text    
    try: 
        temp_text = (temp_text[re.search("title=", temp_text).end():re.search("</a>", temp_text).start()]).split(">")[-1]
        return temp_text
    except:
        return None

def extract_course_title(input_text, working=True):
    """
    extract the following:
    - course title
    - course URL
    """
    try:
        start_index =  re.search("""<h5 class="card-title""", input_text).start()
        end_index = re.search("""<!-- Upcoming Course Run -->""", input_text).start()
    except: return None
        
    temp_text = input_text[start_index:end_index]
    if working:
        return temp_text
    try:
        course_title = temp_text[re.search("title=",temp_text).end():re.search(">\n",temp_text).start()]
        course_title = course_title[1:-1]
    except:
        course_title = None
    try:
        course_url = temp_text[re.search("a href=", temp_text).end():re.search(" target=", temp_text).start()]
        course_url = course_url[1:-1]
    except:
        course_url = None
    return (course_title, course_url)

def extract_course_run(input_text, working=True):
    try:
        start_index =  re.search("""<label><span>Upcoming Course Date:</span></label>""", input_text).start()
        end_index = re.search("""<div class="card-course-sub-info-holder">""", input_text).start()
    except: return None
    temp_text = input_text[start_index:end_index]
    if working:
        return temp_text
    try:
        course_run = temp_text[re.search("""Course_Start_Date_Nearest""",temp_text).end():re.search("</strong>",temp_text).start()][3:]
        return course_run
    except:
        return None
        
def extract_course_duration(input_text, working=True):
    try:
        start_index =  re.search("""<div class="card-course-duration-holder small"><span data-bind="text: Course_Duration">""", input_text).start()
        end_index = re.search("""<!-- ko if: Mode_of_Training_text.length >""", input_text).start()
    except: return None
    temp_text = input_text[start_index:end_index]
    if working:
        return temp_text    
    try:
        course_duration = temp_text[re.search("""Course_Duration""", temp_text).end():re.search("</span>",temp_text).start()][2:]
        return course_duration
    except:
        return None

def extract_mode_of_training(input_text, working=True):
    try:
        start_index =  re.search("""<span data-bind="text: Mode_of_Training_text">""", input_text).start()
        temp_text = input_text[start_index:]
    except:
        return None
    if working:
        return temp_text        
    try:
        course_mode_of_training = temp_text[re.search("""Mode_of_Training_text""", temp_text).end():re.search("</span>",temp_text).start()][2:]
        return course_mode_of_training
    except:
        return None

def extract_full_course_fee(input_text, working=True):
    try:
        start_index =  re.search("""Full Course Fee:""", input_text).start()
        end_index = re.search("""After SkillsFuture Funding:""", input_text).start()
        temp_text = input_text[start_index:end_index]
    except:
        return None
    if working:
        return temp_text    
    try:
        full_course_fee = temp_text[re.search("""Tol_Cost_of_Trn_Per_Trainee""", temp_text).end():re.search("</strong>", temp_text).start()][2:]
        return full_course_fee
    except:
        return None

def extract_subsidised_course_fee(input_text, working=True):
    try:
        start_index =  re.search("""After SkillsFuture Funding:""", input_text).start()
        temp_text = input_text[start_index:]
    except:
        return None
    if working:
        return temp_text  
    try:
        subsidised_course_fee = temp_text[re.search("""EXT_Course_Ref_Nos""", temp_text).end():re.search("</strong>", temp_text).start()][26:]
        return subsidised_course_fee
    except:
        return None


In [18]:
# extract card information first
card_info_list = []
match_indices = []
for match in re.finditer(r"""<div class="card">""", target_text):
    match_indices.append(match.start())
    
for i in range(len(match_indices)):
    if i<len(match_indices)-1:
        card_info_list.append(target_text[match_indices[i]:match_indices[i+1]])
    else:
        card_info_list.append(target_text[match_indices[i]:match_indices[i]+20000])

card_info_list = [x for x in card_info_list if "Institution" in x]
my_pdf = pd.DataFrame({"whole_card_info": card_info_list})

In [19]:
my_pdf["card_body"] = my_pdf["whole_card_info"].map(lambda x: extract_body(x))
my_pdf["card_footer"] = my_pdf["whole_card_info"].map(lambda x: extract_footer(x))

# my_pdf["course_provider_working"] = my_pdf["card_body"].map(lambda x: extract_course_provider(x))
my_pdf["course_provider"] = my_pdf["card_body"].map(lambda x: extract_course_provider(x, False))

# my_pdf["course_title_working"] = my_pdf["card_body"].map(lambda x: extract_course_title(x))
my_pdf["course_title_url"] = my_pdf["card_body"].map(lambda x: extract_course_title(x, False))
my_pdf[["course_title", "course_url"]] = my_pdf["course_title_url"].values.tolist()

# my_pdf["course_run_working"] = my_pdf["card_body"].map(lambda x: extract_course_run(x))
my_pdf["course_run"] = my_pdf["card_body"].map(lambda x: extract_course_run(x, False))

# my_pdf["course_duration_working"] = my_pdf["card_body"].map(lambda x: extract_course_duration(x))
my_pdf["course_duration"] = my_pdf["card_body"].map(lambda x: extract_course_duration(x, False))

# my_pdf["course_mode_of_training_working"] = my_pdf["card_body"].map(lambda x: extract_mode_of_training(x))
my_pdf["course_mode_of_training"] = my_pdf["card_body"].map(lambda x: extract_mode_of_training(x, False))

# my_pdf["full_course_fee_working"] = my_pdf["card_footer"].map(lambda x: extract_full_course_fee(x))
my_pdf["full_course_fee"] = my_pdf["card_footer"].map(lambda x: extract_full_course_fee(x, False))

# my_pdf["subsidised_course_fee_working"] = my_pdf["card_footer"].map(lambda x: extract_subsidised_course_fee(x))
my_pdf["subsidised_course_fee"] = my_pdf["card_footer"].map(lambda x: extract_subsidised_course_fee(x, False))

In [ ]:
FINAL_COLUMNS = ["course_provider", "course_title", "course_url", "course_run", "course_duration", "course_mode_of_training", "full_course_fee", "subsidised_course_fee"]
my_pdf = my_pdf[FINAL_COLUMNS]
my_pdf = my_pdf[my_pdf["course_title"].notnull()].reset_index(drop=True)
my_pdf.to_csv("skillsfuture_cleaned.csv", index=False)

example questions:
- `"is there a course for drone photography?"`
- `"what course should i take if i know a bit of cooking and want to try more advanced cooking methods?"`
- `"i have taken xxx level one before, what should i take now?"`
- `"i only have a budget of 100 on to pof skillsfuture credits, what courses can i take regarding birding?"`
- `"i want to upskill and find jobs that are in demand now. What courses should i take?"`
- `"im an insurance agent with twelve years in the field. what course can i take to upskill to help in my career?"`

model flow:
- document loading
   - [DONE] extract all courses
   - [DONE] clean course information
   - [DONE] randomly select 1000 courses and extract course information from each of the individual webpages
   - [DONE] clean course information
   - [Done] each document will contain the course information for each of the 1000 courses in a specific format
- load into chroma
   - [Done] splitting and chunking
   - [DONE] storing
- querying
   - [DOne] retrieval
   - [Done] generate response
   - improve the prompt template and introduce filtering
- frontend
   - frontend
- deploy
   - deploy

flow:
1. constraint the documents available to the llm based on the following constraints:
   - duration constraint
   - commitment constraint
   - budget constraint
1. using the input, use query methods (e.g. reranker etc.) to retrieve courses that are relevant
1. prepare output

In [ ]:
skillsfuture_pdf = pd.read_csv("../skillsfuture_cleaned.csv")
def clean_course_title(input_str):
    course_title = input_str
    for invalid_token in [":", "/", "@"]:
        course_title = course_title.replace(invalid_token, " ")
    return course_title
skillsfuture_pdf["course_title_cleaned"] = skillsfuture_pdf["course_title"] .map(lambda x: clean_course_title(x))

extract
- number of hours
    - start_index : `<label>Training Commitment</label>`
    - end_index : `courseDetail.modeOfTrainings()`
- conducted in 
    - start_index : `<label>Conducted in</label>`
    - end_index : `mediumOfInstructions()`
- course objective
    - start_index : `<span>About This Course</span>`
    - end_index : `<span>What You'll Learn</span>`
- What you'll learn
    - start_index : `<span>What You'll Learn</span>`
    - end_index : `entryRequirement()`
- Minimum entry requirement
    - start_index : `<span>Minimum Entry Requirement</span>`
    - end_index : `<span>Search by Keyword Tags</span>`

In [160]:
def extract_number_of_hours(input_text, working=True):
    try:
        temp_text = input_text[re.search("<label>Number of Hours</label>", input_text).start():]
        temp_text = temp_text[:re.search("</div>", temp_text).start()]
    except:
        return None
    if working:
        return temp_text
    try:
        temp_text = temp_text[re.search(r"totalTrainingDurationHour()", temp_text).end():][5:]
        return temp_text
    except:
        return None

def extract_conducted_language(input_text, working=True):
    try:
        temp_text = input_text[re.search("<label>Conducted in</label>", input_text).start():]
        temp_text = temp_text[:re.search("</span>", temp_text).start()]
    except:
        return None
    if working:
        return temp_text
    try:
        temp_text = temp_text[re.search(r"text : description", temp_text).end():][2:]
        return temp_text
    except:
        return None
    
def extract_course_objective(input_text, working=True):
    try:
        temp_text = input_text[re.search("<span>About This Course</span>", input_text).start():]
        temp_text = temp_text[:re.search(r"</p>", temp_text).start()]
    except:
        return None
    if working:
        return temp_text
    try:
        temp_text = temp_text[re.search("courseObjective", temp_text).end():][2:]
        return temp_text.replace("\n", " ")
    except:
        return None
    
def extract_learning_outcomes(input_text, working=True):
    try:
        temp_text = input_text[re.search("<span>What You'll Learn</span>", input_text).start():]
        temp_text = temp_text[:re.search(r"</p>", temp_text).start()]
    except:
        return None
    if working:
        return temp_text
    try:
        temp_text = temp_text[re.search("courseContent", temp_text).end():][2:]
        return temp_text
    except:
        return None
    
def extract_entry_req(input_text, working=True):
    try:
        start_index = re.search("<span>Minimum Entry Requirement</span>", input_text).start()
        end_index = re.search("<span>Search by Keyword Tags</span>", input_text).start()
    except:
        return None
    temp_text = input_text[start_index:end_index]
    if working:
        return temp_text
    try:
        temp_text = temp_text[re.search("entryRequirement", temp_text).end():re.search(r"</p>", temp_text).start()][2:]
        return temp_text
    except:
        return None
    


In [ ]:
project_documents_list = [x for x in os.listdir("Project Documents") if x.endswith(".html")]
len(project_documents_list)
course_desc_list = []
for _document in project_documents_list:
    with open(rf"Project Documents/{_document}", "r", encoding='utf-8') as f:
        course_desc = f.read()
    course_desc_list.append(course_desc)
course_desc_pdf = pd.DataFrame({"raw_course_desc": course_desc_list})
course_desc_pdf["course_title_cleaned"] = project_documents_list

# course_desc_pdf["course_number_of_hours_working"] = course_desc_pdf["raw_course_desc"].map(lambda x: extract_number_of_hours(x))
course_desc_pdf["course_number_of_hours"] = course_desc_pdf["raw_course_desc"].map(lambda x: extract_number_of_hours(x, False))

# course_desc_pdf["course_language_working"] = course_desc_pdf["raw_course_desc"].map(lambda x: extract_conducted_language(x))
course_desc_pdf["course_language"] = course_desc_pdf["raw_course_desc"].map(lambda x: extract_conducted_language(x, False))

# course_desc_pdf["course_objective_working"] = course_desc_pdf["raw_course_desc"].map(lambda x: extract_course_objective(x))
course_desc_pdf["course_objective"] = course_desc_pdf["raw_course_desc"].map(lambda x: extract_course_objective(x, False))

# course_desc_pdf["course_learning_outcomes_working"] = course_desc_pdf["raw_course_desc"].map(lambda x: extract_learning_outcomes(x))
course_desc_pdf["course_learning_outcomes"] = course_desc_pdf["raw_course_desc"].map(lambda x: extract_learning_outcomes(x, False))

# course_desc_pdf["course_entry_req_working"] = course_desc_pdf["raw_course_desc"].map(lambda x: extract_entry_req(x))
course_desc_pdf["course_entry_req"] = course_desc_pdf["raw_course_desc"].map(lambda x: extract_entry_req(x, False))

In [ ]:
course_desc_pdf = course_desc_pdf[["course_title_cleaned", "course_number_of_hours", "course_language", "course_objective", "course_learning_outcomes", "course_entry_req"]]
course_desc_pdf["course_title_cleaned"] = course_desc_pdf["course_title_cleaned"].map(lambda x: x[:-5])

In [ ]:
final_pdf = skillsfuture_pdf.merge(course_desc_pdf, on=["course_title_cleaned"], how="inner").drop_duplicates(subset=["course_title_cleaned"]).reset_index(drop=True)
final_pdf.to_csv("skillsfuture_full.csv", index=False)

# Preparing to load into database

In [26]:
final_pdf = pd.read_csv("skillsfuture_full.csv")
FULL_DESC_FIELDS = [x for x in final_pdf.columns if x not in ["course_title_cleaned", "full_course_desc"]]
def form_document_content(row):
    document_content = ""
    for _col in FULL_DESC_FIELDS:
        if isinstance(row[_col], str):
            document_content += rf"""# {_col.replace("_", " ")}\n"""
            document_content += row[_col]
            document_content += r"\n\n"
    return document_content
final_pdf["full_course_desc"] = final_pdf.apply(lambda row: form_document_content(row), axis=1)